In [1]:
import pandas as pd
import numpy as np
import math
import urllib
import requests
import asyncio
from bs4 import BeautifulSoup as BS

In [ ]:
#async functions

def snag_shit_pandas(url):
    response = pd.read_html(url)
    return response

async def snag_shit_soups(url):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    response = soup.find_all('a') 
    return await response


#asynchronous  info
# Get a reference to the event loop
loop = asyncio.get_event_loop()

# Create the list of shit we want to get asnyc
requests = [asyncio.ensure_future(snag_shit_pandas(someDumbassURL)),
            asyncio.ensure_future(snag_shit_pandas(someOtherDumbShitURL))]

# Run the requests
responses = loop.run_until_complete(asyncio.gather(*requests))

responses

#found this dope shit too. add it if ou want to quit long stuff
try:
    loop.run_forever()
except KeyboardInterrupt:
    # Canceling pending tasks and stopping the loop
    asyncio.gather(*asyncio.Task.all_tasks()).cancel()
    # Stopping the loop
    loop.stop()
    # Received Ctrl+C
    loop.close() 

In [ ]:
#--------

In [2]:
#inputs stuff

#tickers = ['OPHT', 'MNTA']
tickers = ['MNTA', 'AON', 'OPHT', 'IIVI', 'MMM', 'GE']
filing_input_type = 'DEF+14'
table_var = "Summary Compensation Table"
period = 2

In [3]:
url_list = []

for i in range(0, len(tickers)):
    url_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers[i] + '&owner=include&action=getcompany&type=' + filing_input_type)    
url_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=MNTA&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AON&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=OPHT&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=IIVI&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=MMM&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=GE&owner=include&action=getcompany&type=DEF+14']

In [4]:
#BS grabs links here
tag_list = []
document_links = []

for j in range(0,len(url_list)):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    doc_link = soup.find_all('a')
    count = 0

    for i in range(0,len(doc_link)):
        x = doc_link[i].get('href')
        y = x.split('/')
        if len(y) > 1:
            if (y[1] == 'Archives' and count < period):
                tag_list.append([tickers[j],x])
                count = count + 1

    for k in range(0,len(tag_list)):
        index_page = requests.get('https://www.sec.gov'+tag_list[k][1])
        soup2= BS(index_page.content, 'lxml')
        doc_link2 = soup2.find_all('a')
        if tag_list[k][0] == tickers[j]:
            document_links.append([tickers[j],doc_link2[8].get('href')])
    
document_links

[['MNTA',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'],
 ['AON', '/Archives/edgar/data/315293/000119312518138792/d471390ddef14a.htm'],
 ['AON', '/Archives/edgar/data/315293/000104746917003055/a2231270zdef14a.htm'],
 ['OPHT',
  '/Archives/edgar/data/1410939/000141093918000020/proxystatementfor2018annua.htm'],
 ['OPHT',
  '/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm'],
 ['IIVI', '/Archives/edgar/data/820318/000119312517286341/d434985ddef14a.htm'],
 ['IIVI', '/Archives/edgar/data/820318/000119312516712280/d245400ddef14a.htm'],
 ['MMM',
  '/Archives/edgar/data/66740/000120677418000890/threem3251561-def14a.htm'],
 ['MMM',
  '/Archives/edgar/data/66740/000120677417000922/threem3172441-def14a.htm'],
 ['GE', '/Archives/edgar/data/40545/000120677418000752/ge3334621-def14a.htm'],
 ['GE', '/Archives/edgar/data/40545/000120677417000738/ge3179831-def14a.htm'

In [6]:
filing_reader = []
for i in range(0,len(document_links)):
    url = 'https://www.sec.gov'+document_links[i][1]
    filing_reader.append([document_links[i][0],pd.read_html(url)])

In [18]:
filing_reader
#filing_reader[0][1][21]

[['MNTA', [                                              0   1  \
   0                                           NaN NaN   
   1                     Filed by the Registrant ý NaN   
   2  Filed by a Party other than the Registrant o NaN   
   3                    Check the appropriate box: NaN   
   4                                             o NaN   
   5                                             o NaN   
   6                                             ý NaN   
   7                                             o NaN   
   8                                             o NaN   
   
                                                      2  
   0                                                NaN  
   1                                                NaN  
   2                                                NaN  
   3                                                NaN  
   4                        Preliminary Proxy Statement  
   5  Confidential, for Use of the Commission Only (...  
  

In [8]:
len(filing_reader)

12

In [32]:
#let's find the right table - if no TOC

totes_scores = []
filings_scores = []
table_location = []


#does it make sense to include "officer" ?
searchParameters = ("Name|Officer|Position|Salary|Stock|Award|Option|Non‑Equity Incentive Plan|Compensation|All Other|Total|Year")

for k in range(0, len(filing_reader)):
    for j in range(0,len(filing_reader[k][1])):
        interim = filing_reader[k][1][j].dropna(how = 'all', axis = 1).dropna(how = 'all', axis = 0).reset_index(drop=True)
        interim.columns = range(interim.shape[1])
        scorer = []

        for i in range(0,interim.shape[0]):
            scorer.append(interim.iloc[i].str.contains(searchParameters).sum())

        totes_scores.append(sum(scorer))
    
    location = totes_scores.index(max(totes_scores))
    table_location.append(location)
    totes_scores = []






In [33]:
#----------- Works to here

table_location


[21, 128, 72, 82, 4, 4, 26, 26, 142, 187, 143, 227]

In [46]:
TOC

,0,1
0,PAGE,ITEM
1,Preceding,Notice of Annual Meeting of Shareholders
2,1,Introduction
3,5,"Matters of Business, Votes Needed and Recommen..."
4,6,PROPOSAL ONE – ELECTION OF DIRECTORS
5,8,Class Three Directors Standing for Election
6,9,Continuing Directors
7,11,Meetings and Standing Committees of the Board
8,12,Director Independence and Corporate Governance...
9,16,Director Compensation in Fiscal Year 2017


In [90]:
#BS grabs proxy hyperlinks - need to add phase out if there are none!

TOC_Link = []

for j in range(0,len(document_links)):
    page = requests.get('https://www.sec.gov'+document_links[j][1])
    soup = BS(page.content, 'lxml')
    prox_link = soup.find_all('a')

    for i in range(0,len(prox_link)):
        check = str(prox_link[i])
        if check.find(table_var) != -1:
            TOC_Link.append([document_links[j][0],prox_link[i]])


    
    

In [93]:
len(TOC_Link)
TOC_Link

[['OPHT',
  <a href="#sbfa4c48ea2c646c5bf2285477391d60e" style="font-family:inherit;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;">Summary Compensation Table</font></a>],
 ['OPHT',
  <a href="#s4d12ffc4ea41466c85a61d9aee3e6503" style="font-family:inherit;font-size:10pt;"><font style="font-family:inherit;font-size:10pt;">Summary Compensation Table</font></a>],
 ['OPHT',
  <a href="#ds46201_summary_compensation_table"><p style="font-family:times;margin-left:20pt;text-indent:-10pt;"><font size="2"> </font><font size="2">Summary Compensation Table</font></p></a>],
 ['OPHT',
  <a href="#dy46201_summary_compensation_table"><p style="font-family:times;margin-left:20pt;text-indent:-10pt;"><font size="2"> </font><font size="2">Summary Compensation Table</font></p></a>],
 ['IIVI', <a href="#toc434985_29">Summary Compensation Table</a>],
 ['IIVI', <a href="#toc245400_30">Summary Compensation Table</a>],
 ['MMM', <a href="#d325156a071">2017 Summary Compensation Table</a>],
 ['M

In [97]:
newthing = pd.read_html('https://www.sec.gov/Archives/edgar/data/820318/000119312517286341/d434985ddef14a.htm#toc434985_29', match = table_var)
type(newthing)

list